In [1]:
import numpy
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
# fix random seed for reproducibility
numpy.random.seed(7)

Using TensorFlow backend.


In [2]:
# load the dataset but only keep the top n words, zero the rest
top_words = 500
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=top_words)

In [3]:
# truncate and pad input sequences
max_review_length = 50
X_train = sequence.pad_sequences(X_train, maxlen=max_review_length)
X_test = sequence.pad_sequences(X_test, maxlen=max_review_length)

In [4]:
# create the model
embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=3, batch_size=64)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 50, 32)            16000     
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 69,301
Trainable params: 69,301
Non-trainable params: 0
_________________________________________________________________
None
Train on 25000 samples, validate on 25000 samples
Epoch 1/3
25000/25000 [==============================] - 31s 1ms/step - loss: 0.5532 - acc: 0.7059 - val_loss: 0.5143 - val_acc: 0.7452
Epoch 2/3
25000/25000 [==============================] - 32s 1ms/step - loss: 0.4930 - acc: 0.7587 - val_loss: 0.4810 - val_acc: 0.7668
Epoch 3/3
25000/25000 [==============================] - 32s 1ms/step - los

In [5]:
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 76.86%


In [25]:
from keras import backend as K

# with a Sequential model
get_0th_layer_output = K.function([model.layers[0].input],
                                  [model.layers[0].output])
layer_output = get_0th_layer_output([X_train])

In [39]:
layer_output[0][0]

array([[ 0.05127354,  0.03347518,  0.05233604, ...,  0.05091812,
        -0.01645943,  0.01821958],
       [ 0.05127354,  0.03347518,  0.05233604, ...,  0.05091812,
        -0.01645943,  0.01821958],
       [ 0.05127354,  0.03347518,  0.05233604, ...,  0.05091812,
        -0.01645943,  0.01821958],
       ..., 
       [-0.0252345 , -0.03148654,  0.01719919, ...,  0.02275859,
         0.031842  , -0.04077763],
       [ 0.00042993,  0.03595172, -0.00697326, ...,  0.05323707,
         0.05543623,  0.01107716],
       [ 0.00317806, -0.01906356, -0.00275898, ..., -0.00741136,
        -0.02303704, -0.01400802]], dtype=float32)